In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

Imported GOOGLE_API_KEY from Kaggle Secrets

In [ ]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.agents import Agent, SequentialAgent
from google.adk.runners import Runner

print("✅ ADK components imported successfully.")

In [ ]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

In [ ]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")
    USER_ID = 123
    MODEL_NAME = "Geminii"
    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ): print(f"{MODEL_NAME} > ", event.content.parts[0].text)
                        
    else:
        print("No queries!")

#Helper Function for Persistent Session Management 
print("✅ Helper functions defined.")

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:
def get_protien_required_perWeek(
    weight: int,
    activity_level: str,
    gender: str,
    city: str,
    dietary_pref: str
) -> dict:
    """
    Calculates protien required per week for a user given weight and activity level.

    This tool calculates protein required per week for the user based on:
    - weight (mandatory)
    - activity level (optional)

    Args:
        weight (int): User's weight in kilograms.
        activity_level (str): Activity level string ("intense", "active", etc.).
        gender (str): User's gender.
        city (str): User's city.
        dietary_pref (str): Dietary preference.

    Returns:
        dict: Protien information result.

        Success:
            {
                "status": "success",
                "protien_per_week": 68
            }

        Error:
            {
                "status": "error",
                "error_message": "User weight not found"
            }
    """
    # Default multiplier for protein per kg per day
    fact = 1.0

    # Determine activity multiplier
    # Adjust factor based on activity level
    # Use more protein if activity is high/intense
    if activity_level and activity_level.lower() == "intense":
        fact = 2.0
    elif activity_level and activity_level.lower() == "active":
        fact = 1.4

    # Validate required values
    if weight is None:
        return {
            "status": "error",
            "error_message": "User weight not found"
        }

    # Protein calculations
    protien_per_day = fact * weight
    protien_per_week = protien_per_day * 7.0

    return {
        "status": "success",
        "protien_per_week": protien_per_week,
        "weight": weight,
        "activity_level": activity_level,
        "gender": gender,
        "city": city,
        "dietary_pref": dietary_pref
    }


print("✅ function created")


In [ ]:
#Initializer Agent that connects to Tool and Calculate Weekly protien intake for user
initializer_agent = LlmAgent(
    name="initializer_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart mediator assistant.

    Meal Planning requests:
    1. Get user details provided from the user like weight, gender, activity_level, city,
      dietary_pref
    2. Fed all details to get_protien_required_perWeek tool in required format.
    3. Get the output String from the tool

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_protien_required_perWeek],
    output_key="user_details"
)



In [ ]:
#Agent Plans meal consider protien and user info 
meal_planner_agent = Agent(
    name="meal_planner_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The `{user_details}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this ouput strictly: {user_details}
    Use Google Search for Planning a meal for week for user considering user details from user_details like protien_per_week required,
    activity_level, gender, dietary_pref, city.
    Also provide user_details in output""",
    tools=[google_search],
    output_key="meal_planning_details"  # The result of this agent will be stored with this key.
)


In [ ]:
#Agent takes input and meal finds ingredients and items. Locates as per user city shops
shopping_agent = Agent(
    name="shopping_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This agent receives the `{meal_planning_details}` from the meal planner agent's output.
    instruction="""Add to this ouput: {meal_planning_details}
    Your task is get city details, dietary_pref and meal planned details. Analyze the ingredients and items required for all
    meals planned for user and use Google Search add shops and store details from users city where he can get all items.
    Format the output bulled and numbered points as required
    Keep it info short crisp, summarize and to point for user to understand easily.""",
    tools=[google_search],
    output_key="final_output"  # This is the final output of the entire pipeline.
)


In [ ]:
root_agent = SequentialAgent(
    name="MealPlannerPipeline",
    sub_agents=[initializer_agent, meal_planner_agent, shopping_agent],
)

print("✅ Sequential Agent created.")

In [ ]:
# Running Agents with Persistent Session 
APP_NAME = "Meal_Planner"
# SQLite database will be created automatically
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
await run_session(
    runner,
    [ "Plan a meal for me for a week considering my details as follows:weight: 75 Kg, activity_level: Active,  gender: male, city:Mumbai ,dietary_pref: vegetarian"],
    "stateful-agentic-session"
)  


In [ ]:
import sqlite3

#Checking for sessions stored
def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()